In [1]:
from sklearn.model_selection import KFold, ParameterGrid
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout
import tensorflow as tf
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

2023-08-09 22:02:07.571334: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-09 22:02:07.674987: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-09 22:02:07.676476: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-09 22:02:08.721388: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [9]:
dataset = "L_540_2022_CERB"
results_path = Path("../results/") / dataset
data = pd.read_csv(results_path / (dataset + "_preprocessed.csv"))
if "file" in data.columns:
    X = data.drop(columns=["label", "file"])    
else:
    X = data.drop(columns=[""])
wns = np.asarray(X.columns.astype(float))
X = np.asarray(X)
#y = pd.factorize(data.label)[0]

Y = pd.factorize(data.label)[0]
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
#y_train = to_categorical(y_train)
#y_test = to_categorical(y_test)

In [10]:
# Use scikit-learn to grid search the batch size and epochs
import numpy as np
import tensorflow as tf
from sklearn.model_selection import GridSearchCV
from scikeras.wrappers import KerasClassifier
# Function to create model, required for KerasClassifier
def create_model():
	# create model
	model = Sequential()
	model.add(Dense(8, input_dim=X_train.shape[1], activation='relu')),
    model.add(Dense(8, activation='relu')),
    #Dropout(0.5),  
    model.add(Dense(y_train.shape[1], activation='softmax'))
	# Compile model
	model.compile(loss='cate_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

# create model
model = KerasClassifier(model=create_model, verbose=0)
# define the grid search parameters
batch_size = [10, 20, 40, 60, 80, 100]
epochs = [10, 50, 100]
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, Y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

2023-08-09 22:48:24.000494: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-08-09 22:48:24.416807: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-08-09 22:48:24.473017: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-08-09 22:48:24.477255: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-08-09 22:48:24.502544: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-08-09 22:48:24.537693: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-08-09 22:48:24.541559: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-08-09 22:48:24.544286: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-08-

ValueError: 
All the 54 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "/apps/anaconda3/envs/py3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/fhwn.ac.at/106374/.local/lib/python3.9/site-packages/scikeras/wrappers.py", line 1494, in fit
    super().fit(X=X, y=y, sample_weight=sample_weight, **kwargs)
  File "/home/fhwn.ac.at/106374/.local/lib/python3.9/site-packages/scikeras/wrappers.py", line 762, in fit
    self._fit(
  File "/home/fhwn.ac.at/106374/.local/lib/python3.9/site-packages/scikeras/wrappers.py", line 931, in _fit
    self._fit_keras_model(
  File "/home/fhwn.ac.at/106374/.local

In [3]:
# Define the number of folds for your CV
n_folds = 3
kf = KFold(n_splits=n_folds)
# Initialize list for storing results
results = []
# Loop over the parameters
for params in param_grid:
    # Initialize list for storing fold scores
    fold_scores = []
    # Perform cross-validation
    for train, test in kf.split(X_train, y_train):
        # Define the model with the current parameters
        model = Sequential([
            Dense(params['neurons'], input_dim=X_train.shape[1], activation='relu', 
                  kernel_regularizer=tf.keras.regularizers.l2(0.001)), BatchNormalization(), Dropout(params['dropout_rate']),
            Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)), Dropout(0.5),
            Dense(16, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)), Dropout(0.5),
            Dense(y_one_hot.shape[1], activation='softmax')  # Number of neurons equals number of classes
        ])
        # Compile model with a custom learning rate
        opt = tf.keras.optimizers.Adam(learning_rate=0.001)
        model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
        # Set up early stopping callback
        early_stopping = EarlyStopping(patience=500)
        # Fit model
        model.fit(X_train[train], y_train[train], 
                  validation_data=(X_test, y_test),
                  epochs=150, 
                  batch_size=params['batch_size'], 
                  verbose=False, 
                  callbacks=[early_stopping])
        # Evaluate model performance
        test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=2)
        # Store fold score
        fold_scores.append(test_accuracy)
    # Compute mean score over all folds and store result
    mean_score = np.mean(fold_scores)
    results.append((params, mean_score))

8/8 - 0s - loss: 1.1571 - accuracy: 0.5208 - 239ms/epoch - 30ms/step
8/8 - 0s - loss: 1.3282 - accuracy: 0.4875 - 228ms/epoch - 29ms/step
8/8 - 0s - loss: 1.1335 - accuracy: 0.5292 - 233ms/epoch - 29ms/step
8/8 - 0s - loss: 1.1485 - accuracy: 0.5458 - 233ms/epoch - 29ms/step
8/8 - 0s - loss: 1.0963 - accuracy: 0.5875 - 239ms/epoch - 30ms/step
8/8 - 0s - loss: 1.1264 - accuracy: 0.5792 - 227ms/epoch - 28ms/step
8/8 - 0s - loss: 1.1704 - accuracy: 0.5333 - 232ms/epoch - 29ms/step
8/8 - 0s - loss: 1.0918 - accuracy: 0.5958 - 232ms/epoch - 29ms/step
8/8 - 0s - loss: 1.2449 - accuracy: 0.4917 - 235ms/epoch - 29ms/step
8/8 - 0s - loss: 1.0636 - accuracy: 0.6042 - 242ms/epoch - 30ms/step
8/8 - 0s - loss: 1.1153 - accuracy: 0.5167 - 230ms/epoch - 29ms/step
8/8 - 0s - loss: 1.1459 - accuracy: 0.5917 - 235ms/epoch - 29ms/step
8/8 - 0s - loss: 1.1298 - accuracy: 0.5333 - 238ms/epoch - 30ms/step
8/8 - 0s - loss: 1.0930 - accuracy: 0.5583 - 237ms/epoch - 30ms/step
8/8 - 0s - loss: 1.1620 - accuracy

In [4]:
# Sort the results by score and print the best parameters
results.sort(key=lambda x: x[1], reverse=True)
best_params, best_score = results[0]
print(f'Best parameters: {best_params}')
print(f'Best score: {best_score}')

Best parameters: {'batch_size': 32, 'dropout_rate': 0.5, 'neurons': 128}
Best score: 0.6152777671813965
